In [ ]:
import pandas as pd
import numpy as np
import mysql.connector
import matplotlib.pyplot as plt

In [ ]:
class DataExtrator:
    def __init__(self):
        self.mydb = mysql.connector.connect(
            host="192.168.50.100",
            user="root",
            password="Admin1234!",
            database="web_info"
        )
        self.mycursor = self.mydb.cursor()

    def getRequestInfo(self):
        sqlStr = "select name_id, name from request_info"
        self.mycursor.execute(sqlStr)
        return pd.DataFrame(self.mycursor.fetchall(), columns=['name_id', 'name'])

    def getWebInfo(self, name_id):
        columnsName = ['name_id', 'create_time', 'weibo_follower', 'v_index_wenyu', 'chaohua_read_yi', 'chaohua_count_wan', 'bili_channel_view_count_yi',
                       'bili_channel_feature_count', 'douyin_ch_user_count', 'douyin_ch_view_count']
        sqlStr = f"select name_id, create_time, weibo_follower, v_index_wenyu, chaohua_read_yi, chaohua_count_wan, bili_channel_view_count_yi, \
                bili_channel_feature_count, douyin_ch_user_count, douyin_ch_view_count from web_data where name_id = {name_id}"
        self.mycursor.execute(sqlStr)
        return pd.DataFrame(self.mycursor.fetchall(), columns=columnsName)

In [ ]:
request_info_df = DataExtrator().getRequestInfo().sort_values('name_id')
request_info_df.to_csv("request_info.csv", index=False)

In [ ]:
def dataPlotter(accounts=[8], data_name='weibo_follower', need_diff=True, days=30):
    plt.figure(figsize=(40, 10))
    extractor = DataExtrator()
    for account in accounts:
        accountData = extractor.getWebInfo(account)
        normal = accountData[data_name]
        diff = np.concatenate([[None], np.diff(accountData[data_name])])
        if (need_diff):
            plt.plot(accountData['create_time'][-days:], diff[-days:], '-o', label=account)
        else:
            plt.plot(accountData['create_time'][-days:], normal[-days:], '-o', label=account)
        plt.xticks(accountData['create_time'][-days:], rotation=45)
    plt.title(data_name)
    plt.legend()

In [ ]:
dataPlotter(data_name='weibo_follower')
dataPlotter(data_name='chaohua_read_yi')
dataPlotter(data_name='chaohua_count_wan')
# dataPlotter(data_name='bili_channel_view_count_yi')
# dataPlotter(data_name='douyin_ch_user_count')
dataPlotter(data_name='douyin_ch_view_count')